<a href="https://colab.research.google.com/github/URK21CO3003NathanShibuJohn/FinalYearProject/blob/main/DataPreprocessingNew.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# prompt: mount drive

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import scipy.signal as signal
from sklearn.preprocessing import MinMaxScaler

# Load EEG Dataset
ndata = pd.read_excel("/content/drive/MyDrive/NData_labeled_sample.xlsx")

# Adaptive Signal Denoising using LMS Filter
def adaptive_denoising(eeg_signal, reference_signal, mu=0.0001):
    """
    Adaptive noise cancellation using an LMS filter with stability fixes.
    """
    n = len(eeg_signal)
    filtered_signal = np.copy(eeg_signal)
    filter_weights = np.zeros(n)
    reference_signal = (reference_signal - np.mean(reference_signal)) / (np.std(reference_signal) + 1e-6)  # Normalize reference signal

    for t in range(1, n):
        if t < len(reference_signal):
            error = eeg_signal[t] - (filter_weights[t-1] * reference_signal[t])
            filter_weights[t] = np.clip(filter_weights[t-1] + mu * error * reference_signal[t], -1, 1)  # Clamp values
            filtered_signal[t] = eeg_signal[t] - (filter_weights[t] * reference_signal[t])

    return filtered_signal

# Min-Max Normalization
def normalize_signal(eeg_signal):
    """
    Min-Max Normalization to scale EEG signals properly.
    """
    min_val = np.min(eeg_signal)
    max_val = np.max(eeg_signal)
    if max_val - min_val == 0:
        return np.zeros_like(eeg_signal)  # Return zeros if no variation
    return (eeg_signal - min_val) / (max_val - min_val + 1e-6)  # Prevent division by zero

# Apply Preprocessing
raw_eeg = ndata.iloc[:, :-1].values  # EEG Data (excluding label column)
reference_signal = raw_eeg[:, 0]  # Assume first column is reference (EOG/EMG)
preprocessed_eeg = np.apply_along_axis(lambda x: adaptive_denoising(x, reference_signal), axis=0, arr=raw_eeg)
normalized_eeg = np.apply_along_axis(normalize_signal, axis=0, arr=preprocessed_eeg)

# Ensure no NaN or infinite values
normalized_eeg = np.nan_to_num(normalized_eeg, nan=0.0, posinf=1.0, neginf=0.0)

# Update DataFrame
ndata_preprocessed = pd.DataFrame(normalized_eeg, columns=ndata.columns[:-1])
ndata_preprocessed['Label'] = ndata.iloc[:, -1].values

# Save Preprocessed Data
#ndata_preprocessed.to_csv("/mnt/data/NData_Preprocessed.csv", index=False)

print("Preprocessing complete. Data saved to 'NData_Preprocessed.csv'")


Preprocessing complete. Data saved to 'NData_Preprocessed.csv'


In [ ]:
ndata_preprocessed.shape

(10000, 10)

In [ ]:
ndata_preprocessed.head()

,theta band,alpha band,low beta band,high beta band,gamma band,Arousal,Valence,Dominance,Liking,Label
0,0.101552,0.285039,0.197259,0.406761,0.349518,0.313227,0.371184,0.310773,0.296950,Happy
1,0.182281,0.335973,0.313323,0.444702,0.379454,0.313170,0.371116,0.310716,0.296896,Happy
2,0.200539,0.265021,0.234414,0.317285,0.216444,0.313152,0.371095,0.310698,0.296879,Happy
3,0.049593,0.214466,0.236117,0.181055,0.151009,0.312501,0.370323,0.310052,0.296262,Happy
4,0.151873,0.533913,0.285141,0.394164,0.170225,0.312747,0.370615,0.310296,0.296495,Happy


In [ ]:
ndata_preprocessed.to_csv("/content/drive/MyDrive/NData_Preprocessednew.csv", index=False)
